In [1]:
from cryptofeed.defines import TRADES, L2_BOOK, BID, ASK

import pandas as pd
import json

import time
import datetime

import threading
import schedule
import redis

from algos.vol_trend.backtest import perform_backtests
from algos.vol_trend.live_trader import liveTrading

In [2]:
def get_position_balance():
    r = redis.Redis(host='localhost', port=6379, db=0)
    move_backtest = pd.read_csv('data/trades_move.csv')
    perp_backtest = pd.read_csv('data/trades_perp.csv')

    pairs = json.load(open('algos/vol_trend/pairs.json'))
    pairs.append('BTC-PERP')

    details_df = pd.DataFrame()

    for asset in pairs:
        curr_detail = pd.Series()
        curr_detail['name'] = asset
        
        try:
            curr_detail['position'] = r.get('FTX_{}_current_pos'.format(asset)).decode()
        except:
            curr_detail['position'] = "NONE"

        try:
            curr_detail['entry'] = float(r.get('FTX_{}_avgEntryPrice'.format(asset)).decode())
        except:
            curr_detail['entry'] = 0
        
        try:
            curr_detail['pos_size'] = float(r.get('FTX_{}_pos_size'.format(asset)).decode())
        except:
            curr_detail['pos_size'] = 0

        if asset == "BTC-PERP":
            curr_detail['backtest_position'] = 'SHORT' if perp_backtest.iloc[-1]['Type'] == 'SELL' else 'LONG'
            curr_detail['backtest_date'] = perp_backtest.iloc[-1]['Date']
            curr_detail['entry_price'] = round(perp_backtest.iloc[-1]['Price'], 2)
        else:
            curr_df = move_backtest[move_backtest['Data'] == asset]

            curr_detail['backtest_position'] = 'NONE'
            curr_detail['backtest_date'] = ""

            if len(curr_df) > 0:
                if abs(curr_df['Size'].sum()) > 0.01:
                    curr_detail['backtest_position'] = 'SHORT' if curr_df.iloc[-1]['Type'] == 'SELL' else 'LONG'
                    curr_detail['backtest_date'] = curr_df.iloc[-1]['Date']
                    curr_detail['entry_price'] = round(curr_df.iloc[-1]['Price'], 2)

        details_df = details_df.append(curr_detail, ignore_index=True)


    balances = {}
    try:
        balances['MOVE_BALANCE'] = round(float(r.get('FTX_MOVE_balance').decode()), 2)
    except:
        balances['MOVE_BALANCE'] = 0

    try:
        balances['PERP_BALANCE'] = round(float(r.get('FTX_PERP_balance').decode()), 2)
    except:
        balances['PERP_BALANCE'] = 0

    return details_df, balances

In [3]:
def get_overriden(details_df):
    replace_perp = ""
    replace_move = ""

    try:
        if float(r.get('override_perp').decode()) == 1:
            perp = float(r.get('perp_override_direction').decode())

        if perp == 1:
            replace_perp = 'LONG'
        elif perp == -1:
            replace_perp = 'SHORT'
        else:
            replace_perp = 'NONE'
    except:
        pass

    try:
        if float(r.get('override_move').decode()) == 1:
            move = float(r.get('move_override_direction').decode())

        if move == 1:
            replace_move = 'LONG'
        elif move == -1:
            replace_move = 'SHORT'
        else:
            replace_move = 'NONE'
    except:
        pass

    new_details = pd.DataFrame()
    for idx, row in details_df.iterrows():
        if row['name'] == 'BTC-PERP':
            if replace_perp != "":
                row['backtest_position'] = replace_perp
        else:
            if replace_move != "":
                if row['backtest_position'] != "NONE":
                    row['backtest_position'] = replace_move
                    
        new_details = new_details.append(row, ignore_index=True)

    return new_details

In [4]:
print("Time: {}".format(datetime.datetime.utcnow()))
perform_backtests()

pairs = json.load(open('algos/vol_trend/pairs.json'))
pairs.append('BTC-PERP')

for pair in pairs:
    lt = liveTrading(pair)
    lt.set_position()

/home/warproxxx/Desktop/Projects/algorithms/algos/vol_trend/backtest.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['curr_group'] = curr_group
/home/warproxxx/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


BTC-MOVE-2019Q4
BTC-MOVE-2020Q1
BTC-MOVE-2020Q2
BTC-MOVE-2020Q3
BTC-MOVE-2020Q4
BTC-MOVE-2021Q1
BTC-MOVE-2021Q2


In [5]:
enabled = 1
stop_perp = 0
stop_move = 0

try:
    enabled = float(r.get('vol_trend_enabled').decode())
except:
    pass

try:
    stop_perp = float(r.get('stop_perp').decode())
except:
    pass

try:
    stop_move = float(r.get('stop_move').decode())
except:
    pass

In [ ]:
details_df, balances = get_position_balance()
for idx, row in details_df.iterrows():
    lt = liveTrading(row['name'])
    lt.set_position()

details_df = get_overriden(details_df)

details_df['target_pos'] = details_df['backtest_position'].replace("LONG", 1).replace("SHORT", -1).replace("NONE", 0)
details_df['curr_pos'] = details_df['position'].replace("LONG", 1).replace("SHORT", -1).replace("NONE", 0)

to_close = details_df[details_df['target_pos'] == 0]

for idx, row in to_close.iterrows():
    if row['name'] == 'BTC-PERP':
        curr_stop = stop_perp
    else:
        curr_stop = stop_move

    if curr_stop == 0:
        if row['curr_pos'] != 0:
            lt = liveTrading(symbol=row['name'])
            lt.fill_order('close', row['position'].lower())

to_open = details_df[details_df['target_pos'] != 0]

<ipython-input-2-81f1e477213b>:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  curr_detail = pd.Series()
